In [90]:
import requests
from bs4 import BeautifulSoup
import time
import re
import wikipediaapi
from tqdm import tqdm
import os

https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/People <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/History <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Geography <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Arts <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Philosophy_and_religion <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Everyday_life <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Society_and_social_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Biology_and_health_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Physical_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Mathematics <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Technology <br>
<b> Not Done </b> <br>

In [91]:
url = "https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Technology"

In [92]:
path = "dataset-level-5/" + url.split("/")[-1]

In [93]:
article_path = path + "/Article/"
summary_path = path + "/Summary/"
os.makedirs(path, exist_ok=True)
os.makedirs(article_path, exist_ok=True)
os.makedirs(summary_path, exist_ok=True)

In [94]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

tags = soup.find_all(lambda tag: tag.name == 'a' and 'href' in tag.attrs and 'title' in tag.attrs and len(tag.attrs) == 2)
article_names = [ tag.text for tag in tags]

In [95]:
pattern = '.*\/wiki.*'
success_count = 0
fail_count = 0
wiki_wiki = wikipediaapi.Wikipedia('en',timeout=10.0)

done_files = set()

for i, tag in enumerate( tqdm(tags) ):
    attributes = tag.attrs
    href = attributes["href"]
    title = attributes["title"]
    if re.match(pattern, href) and title not in done_files :
        #print(href + ":" + title)
        
        done_files.add(title)
        
        page_py = wiki_wiki.page(title)
        
        if page_py.exists() == False:
            #print("Failed for {}".format(title))
            fail_count += 1
            
        else:

            if "Category:Wikipedia" in title or "Wikipedia:" in title or "User:" in title or "Help:" in title or "Template talk:" in title or "Template:" in title or "Category:" in title:
                continue

            with open(article_path+"/{}.txt".format(title.replace('/','-')),'w') as file:
                file.write(page_py.text)

            with open(summary_path+"/{}.txt".format(title.replace('/','-')),'w') as file:
                file.write(page_py.summary)
                
            success_count += 1
            
    if i%100 == 0 and i!=0:
        time.sleep(60)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 945/945 [10:55<00:00,  1.44it/s]


In [96]:
print(success_count)

740


In [97]:
print(fail_count)

10
